# DenoiSeg Example: Mouse Nuclei
This is an example notebook which illustrates how DenoiSeg should be trained. In this notebook we use nuclei dataset from a mouse skull from our collaborators. We already split the data into train and test images. From the train images we then extracted 908 training and 160 validation patches of size 128x128. The test set contains 67 images of size 256x256.

In [ ]:
# Here we are just importing some libraries which are needed to run this notebook.

import warnings
warnings.filterwarnings('ignore')

import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage

from denoiseg.models import DenoiSeg, DenoiSegConfig
from denoiseg.utils.misc_utils import combine_train_test_data, shuffle_train_data, augment_data
from denoiseg.utils.seg_utils import *
from denoiseg.utils.compute_precision_threshold import measure_precision
from denoiseg.utils.denoiseg_data_preprocessing import generate_patches_from_list

from csbdeep.utils import plot_history

import urllib
import os
import zipfile

## Downloading and  Data Loading
We created three versions of this dataset by adding Gaussian noise with zero mean and standard deviations 10 and 20. The dataset are marked with the suffixes n0, n10 and n20 accordingly.

In the next cell you can choose which `noise_level` you would like to investigate.

In [ ]:
# Choose the noise level you would like to look at:
# Values: 'n0', 'n10', 'n20'
noise_level = 'n20'

In [ ]:
# create a folder for our data
if not os.path.isdir('./data'):
    os.mkdir('data')
    
if noise_level == 'n0':
    link = 'https://zenodo.org/record/5157001/files/Mouse_n0.zip?download=1'
elif noise_level == 'n10':
    link = 'https://zenodo.org/record/5157003/files/Mouse_n10.zip?download=1'
elif noise_level == 'n20':
    link = 'https://zenodo.org/record/5157008/files/Mouse_n20.zip?download=1'
else:
    print('This noise level does not exist for this dataset.')

# check if data has been downloaded already
zipPath="data/Mouse_{}.zip".format(noise_level)
if not os.path.exists(zipPath):
    #download and unzip data
    data = urllib.request.urlretrieve(link, zipPath)
    with zipfile.ZipFile(zipPath, 'r') as zip_ref:
        zip_ref.extractall("data")

In [ ]:
# Loading of the training images
trainval_data =  np.load('data/Mouse_{}/train/train_data.npz'.format(noise_level))
train_images = trainval_data['X_train'].astype(np.float32)
train_masks = trainval_data['Y_train']
val_images = trainval_data['X_val'].astype(np.float32)
val_masks = trainval_data['Y_val']

In [ ]:
test_data =  np.load('data/Mouse_{}/test/test_data.npz'.format(noise_level))
test_images = test_data['X_test'].astype(np.float32)
test_masks = test_data['Y_test']
plt.imshow(test_images[1])

In [ ]:
print("Shape of train_images: {}".format(train_images.shape))
print("Shape of train_masks:  {}".format(train_masks.shape))
print("Shape of val_images:   {}".format(val_images.shape))
print("Shape of val_masks:    {}".format(val_masks.shape))

## Small Amounts of Annotated Training Data
With DenoiSeg we present a solution to train deep neural networks if only few annotated ground truth segmentations are available. We simulate such a scenario by zeroing out all but a fraction of the available training data. In the next cell you can specify the percentage of training images for which ground truth annotations are available.

In [ ]:
# Set the number of annotated training images.
# Values: 0.0 (no annotated images) to total number of training images (all images have annotations)
number_of_annotated_training_images = 38
assert number_of_annotated_training_images >= 0.0 and number_of_annotated_training_images <=train_images.shape[0]

In [ ]:
# Seed to shuffle training data (annotated GT and raw image pairs).
seed = 1 

# First we shuffle the training images to remove any bias.
X_shuffled, Y_shuffled = shuffle_train_data(train_images, train_masks, random_seed=seed)

# Here we convert the number of annotated images to be used for training as percentage of available training data.
percentage_of_annotated_training_images = float((number_of_annotated_training_images/train_images.shape[0]) * 100.0)
assert percentage_of_annotated_training_images >= 0.0 and percentage_of_annotated_training_images <= 100.0

# Here we zero out all training images which are not part of the 
# selected percentage.
X_frac, Y_frac = zero_out_train_data(train_images, train_masks, fraction=percentage_of_annotated_training_images)

# Here we generate patches from images and apply augmentation
X_final, Y_final = generate_patches_from_list([X_frac], [Y_frac], augment=True, shuffle=False, shape=(128, 128))
X_val_final, Y_val_final = generate_patches_from_list([val_images], [val_masks], augment=False, shape=(128, 128))

X_final = X_final[... ,np.newaxis]
Y_final = convert_to_oneHot(Y_final, n_classes=3)

X_val_final = X_val_final[... ,np.newaxis]
Y_val_final = convert_to_oneHot(Y_val_final, n_classes=3)

print("Shape of X:     {}".format(X_final.shape))
print("Shape of Y:     {}".format(Y_final.shape))
print("Shape of X_val: {}".format(X_val_final.shape))
print("Shape of Y_val: {}".format(Y_val_final.shape))

Next we look at a single sample. In the first column we show the input image, in the second column the background segmentation, in the third column the foreground segmentation and in the last column the border segmentation.

With the parameter `sample` you can choose different training patches. You will notice that not all of them have a segmentation ground truth.

In [ ]:
sample = 0
plt.figure(figsize=(20,5))
plt.subplot(1,4,1)
plt.imshow(X_final[sample,...,0])
plt.axis('off')
plt.title('Raw validation image')
plt.subplot(1,4,2)
plt.imshow(Y_final[sample,...,0], vmin=0, vmax=1, interpolation='nearest')
plt.axis('off')
plt.title('1-hot encoded background')
plt.subplot(1,4,3)
plt.imshow(Y_final[sample,...,1], vmin=0, vmax=1, interpolation='nearest')
plt.axis('off')
plt.title('1-hot encoded foreground')
plt.subplot(1,4,4)
plt.imshow(Y_final[sample,...,2], vmin=0, vmax=1, interpolation='nearest')
plt.axis('off')
plt.title('1-hot encoded border')

### Configure network parameters

In [ ]:
train_batch_size = 128
train_steps_per_epoch = min(400, max(int(X_final.shape[0]/train_batch_size), 10))

In [ ]:
### In the next cell, you can choose how much relative importance (weight) to assign to denoising 
### and segmentation tasks by choosing appropriate value for denoiseg_alpha (between 0 and 1; with 0 being
### only segmentation and 1 being only denoising. Here we choose denoiseg_alpha = 0.5)

In [ ]:
conf = DenoiSegConfig(X_final, unet_kern_size=3, n_channel_in=1, n_channel_out=4, relative_weights = [1.0,1.0,5.0],
                      train_steps_per_epoch=train_steps_per_epoch, train_epochs=10, 
                      batch_norm=True, train_batch_size=train_batch_size, unet_n_first = 32, 
                      unet_n_depth=4, denoiseg_alpha=0.5, train_tensorboard=False)

vars(conf)

In [ ]:
model_name = 'DenoiSeg_Mouse_n20'
basedir = 'models'
model = DenoiSeg(conf, model_name, basedir)

In [ ]:
history = model.train(X_final, Y_final, (X_val_final, Y_val_final))

In [ ]:
history.history.keys()

In [ ]:
plot_history(history, ['loss', 'val_loss'])

## Computing Threshold Value
The network predicts 4 output channels:
1. The denoised input.
2. The foreground likelihoods.
3. The background likelihoods.
4. The border likelihoods.

We will threshold the foreground prediction image to obtain object segmentations. The optimal threshold is determined on the validation data. Additionally we can optimize the threshold for a given measure. In this case we choose the Average Precision (AP) measure.

In [ ]:
threshold, val_score = model.optimize_thresholds(val_images.astype(np.float32), val_masks, measure=measure_precision(), axes='YX')

print("The higest score of {} is achieved with threshold = {}.".format(np.round(val_score, 3), threshold))

## Test Data
Finally we load the test data and run the prediction.

In [ ]:
test_data =  np.load('data/Mouse_{}/test/test_data.npz'.format(noise_level), allow_pickle=True)
test_images = test_data['X_test']
test_masks = test_data['Y_test']

In [ ]:
predicted_denoised, predicted_images, precision_result, _ = model.predict_denoised_label_masks(test_images,
                                                                                               test_masks,
                                                                                               axes='YX',
                                                                                               threshold=threshold,
                                                                                               measure=measure_precision())
print("Average precision over all test images with threshold = {} is {}.".format(threshold, np.round(precision_result, 3)))

### Visualize the results

In [ ]:
sl = 0
fig = plt.figure()
plt.figure(figsize=(20,10))
plt.subplot(1, 3, 1)
plt.imshow(test_images[sl])
plt.title("Raw image")

plt.subplot(1, 3, 2)
plt.imshow(predicted_images[sl])
plt.title("Predicted segmentation")

plt.subplot(1, 3, 3)
plt.imshow(test_masks[sl])
plt.title("Ground truth segmentation")

plt.show()

In [ ]:
print("Number of annotated images used for training:", number_of_annotated_training_images)
print("Noise level:", noise_level)
print("Considered alpha:", conf.denoiseg_alpha)

### Expected results for this dataset

<b>AP scores for Mouse n0 dataset</b>

||5 imgs|10 imgs|19 imgs|38 imgs|76 imgs|
|--- |--- |--- |--- |--- |--- |
|Alpha 0.5|0.414±0.034|0.432±0.021|0.516±0.005|0.523±0.011|0.544±0.006|
|Alpha 0.3|0.433±0.021|0.453±0.022|0.507±0.009|0.513±0.017|0.552±0.008|
|Alpha 0.7|0.437±0.021|0.403±0.018|0.506±0.013|0.539±0.004|0.540±0.011|
|Alpha best|0.462±0.019|0.484±0.020|0.539±0.003|0.546±0.004|0.571±0.003|

<b>AP scores for Mouse n10 dataset</b>

||5 imgs|10 imgs|19 imgs|38 imgs|76 imgs|
|--- |--- |--- |--- |--- |--- |
|Alpha 0.5|0.449±0.025|0.491±0.025|0.515±0.007|0.534±0.010|0.559±0.009|
|Alpha 0.3|0.458±0.021|0.462±0.025|0.502±0.009|0.513±0.009|0.561±0.009|
|Alpha 0.7|0.423±0.014|0.439±0.027|0.529±0.006|0.544±0.007|0.569±0.008|
|Alpha best|0.497±0.021|0.502±0.023|0.558±0.002|0.558±0.006|0.586±0.006|

<b>AP scores for Mouse n20 dataset</b>

||5 imgs|10 imgs|19 imgs|38 imgs|76 imgs|
|--- |--- |--- |--- |--- |--- |
|Alpha 0.5|0.489±0.023|0.495±0.023|0.541±0.011|0.553±0.011|0.560±0.006|
|Alpha 0.3|0.477±0.022|0.465±0.031|0.523±0.006|0.529±0.010|0.545±0.005|
|Alpha 0.7|0.449±0.008|0.481±0.022|0.533±0.011|0.560±0.007|0.581±0.004|
|Alpha best|0.538±0.011|0.524±0.020|0.562±0.004|0.571±0.004|0.591±0.006|

### Export your model for Fiji

In [ ]:
model.export_TF(name='DenoiSeg - Mouse Nuclei Example', 
                description='This is the 2D DenoiSeg example trained on Mouse Nuclei data in python.', 
                authors=["Tim-Oliver Buchholz", "Mangal Prakash", "Alexander Krull", "Florian Jug"],
                test_img=X_val[0,...,0], axes='YX',
                patch_shape=(128, 128))

